In [58]:
import pandas as pd
import tensorflow as tf
from matplotlib import pyplot as plt
import numpy as np
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dropout, Bidirectional, Dense, Embedding
import gradio as gr


In [2]:
df = pd.read_csv('train.csv')
df.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0


In [3]:
vectorizer = TextVectorization(max_tokens=20000, output_mode='int', output_sequence_length=1800)
vectorizer.adapt(df['comment_text'])
features = vectorizer(df['comment_text'])

In [4]:
labels = tf.convert_to_tensor((df.drop(columns=['id', 'comment_text'])).to_numpy())

In [5]:
dataset = tf.data.Dataset.from_tensor_slices((features, labels))

In [6]:
dataset = dataset.cache()
dataset = dataset.shuffle(len(dataset))
dataset = dataset.batch(16)
dataset = dataset.prefetch(8) # helps bottlenecks
train = dataset.take(int(len(dataset)*.7))
val = dataset.skip(int(len(dataset)*.7)).take(int(len(dataset)*.2))
test = dataset.skip(int(len(dataset)*.9)).take(int(len(dataset)*.1))

In [7]:
print(len(train))
print(len(val))
print(len(test))

6981
1994
997


In [8]:
model = Sequential()

model.add(Embedding(20000+1, 32))
model.add(Bidirectional(LSTM(64, kernel_initializer='glorot_uniform')))
model.add(Dense(64, activation='relu'))
model.add(Dense(128, activation='relu'))
model.add(Dense(256, activation='relu'))
model.add(Dense(6, activation='sigmoid'))

model.compile(loss='BinaryCrossentropy', optimizer='Adam')
model.summary()
history = model.fit(train, epochs=1, validation_data=val)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, None, 32)          640032    
                                                                 
 bidirectional (Bidirection  (None, 128)               49664     
 al)                                                             
                                                                 
 dense (Dense)               (None, 64)                8256      
                                                                 
 dense_1 (Dense)             (None, 128)               8320      
                                                                 
 dense_2 (Dense)             (None, 256)               33024     
                                                                 
 dense_3 (Dense)             (None, 6)                 1542      
                                                        

KeyboardInterrupt: 

In [10]:
input_text = vectorizer(['You freaking rule.'])
res = model.predict(input_text)

1/1 [==============================] - 1s 1s/step


In [11]:
for r in res[0]:
    if r < 0.5:
        r = 0
    elif r > 0.5:
        r = 1
    
    print(r)

0
0
0
0
0
0


In [12]:
batch_x = [t[0] for t in train][0]
batch_y = [t[1] for t in train][0]

In [13]:
predictions = model.predict(batch_x)

1/1 [==============================] - 0s 489ms/step


In [70]:
for i in predictions[0]:
    if i < 0.5:
        i = 0
        print(i)

0
0
0
0
0
0


In [71]:
batch_y[0]

<tf.Tensor: shape=(6,), dtype=int64, numpy=array([0, 0, 0, 0, 0, 0], dtype=int64)>

In [64]:
gr.Interface??